<a href="https://colab.research.google.com/github/housemLassoued/ML-deployment/blob/main/Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
import pandas as pd


In [138]:
df=pd.read_csv('/content/Mall_Customers.csv')

In [139]:
df.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [141]:
for column in df.select_dtypes(include=[ 'int64']).columns:
    fig = px.box(df, y=column, title=f'Box plot for {column}')
    fig.show()


In [142]:
df[df['Annual Income (k$)']==137].value_counts()

,,,,,count
CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100),
199,Male,32,137,18,1
200,Male,30,137,83,1


In [143]:
df=df.drop(df[df['Annual Income (k$)']==137].index)

In [144]:
df.shape

(198, 5)

In [145]:
df['Gender'].value_counts()

,count
Gender,
Female,112
Male,86


In [146]:
import plotly.express as px


fig = px.histogram(df, x='Gender', color='Gender', category_orders={"Gender": df['Gender'].value_counts().index})
fig.update_layout(
    width=500,
    height=500,
    title='Count of Genders',
    xaxis_title='Gender',
    yaxis_title='Count'
)
fig.show()



In [147]:
import plotly.express as px


fig = px.histogram(df, x='Age', color='Age', category_orders={"Age": df['Age'].value_counts().index})
fig.update_layout(
    width=500,
    height=500,
    title='Count of Age  ',
    xaxis_title='Age',
    yaxis_title='Count'
)
fig.show()



In [148]:
import plotly.express as px


fig = px.histogram(df, x='Annual Income (k$)', color='Annual Income (k$)', category_orders={"Annual Income (k$)": df['Annual Income (k$)'].value_counts().index})
fig.update_layout(
    width=500,
    height=500,
    title='Count of Annual Income (k$)  ',
    xaxis_title='Annual Income (k$)',
    yaxis_title='Count'
)
fig.show()


In [149]:
df['Gender']=df['Gender'].map({'Male':1,'Female':0})

In [150]:
df.nunique()

,0
CustomerID,198
Gender,2
Age,51
Annual Income (k$),63
Spending Score (1-100),83


In [151]:
import plotly.graph_objects as go



correlation = df.corr()


mask = correlation > 0.8


mask_df = mask.astype(int)


fig = go.Figure(data=go.Heatmap(
    z=mask_df,
    x=mask_df.columns,
    y=mask_df.index,
    colorscale='Viridis',
    showscale=False,
    hoverinfo='text'
))


fig.update_layout(
    title='Heatmap des Corrélations (> 0.8)',
    xaxis_nticks=36,
    yaxis_nticks=36,
    width=800,
    height=800
)

fig.show()


In [152]:

from sklearn.manifold import TSNE


model = TSNE(n_components=2, random_state=0)
tsne_data = model.fit_transform(df)


tsne_df = pd.DataFrame(tsne_data, columns=['TSNE Component 1', 'TSNE Component 2'])


fig = px.scatter(tsne_df, x='TSNE Component 1', y='TSNE Component 2',
                 title='t-SNE Visualization')
fig.show()



In [153]:
from sklearn.cluster import KMeans

errors = []
for k in range(1, 21):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(df)
    errors.append(kmeans.inertia_)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will 

In [154]:
error_df = pd.DataFrame({'Clusters': range(1, 21), 'Error': errors})
error_df

,Clusters,Error
0,1,941667.565657
1,2,373539.555556
2,3,262201.954371
3,4,187496.790930
4,5,149355.243411
5,6,115355.704567
6,7,97709.841213
7,8,82429.190667
8,9,73282.068899
9,10,65220.333660


In [155]:
fig = px.line(error_df, x='Clusters', y='Error', markers=True, title='Elbow Method for Optimal k')
fig.update_traces(marker=dict(size=8, color='red'), line=dict(color='blue'))
fig.show()

In [156]:

model = KMeans(init='k-means++',
			n_clusters=6,
			max_iter=500,
			random_state=22)
clusters = model.fit_predict(df)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [157]:
tsne_df['Cluster'] = clusters

In [158]:
fig = px.scatter(tsne_df, x='TSNE Component 1', y='TSNE Component 2', color='Cluster',
                 title='t-SNE Visualization with K-means Clusters',
                 color_continuous_scale=px.colors.qualitative.G10)  # Utiliser une palette de couleurs distinctes
fig.show()